In [1]:
import pandas as pd
import numpy as np
import os 


In [8]:
import pandas as pd
import os

# Ocean_data 폴더 내의 모든 CSV 파일 목록을 가져옵니다.
folder_path = 'Ocean_data'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('19') or f.startswith('20')]

# 각 파일을 순차적으로 읽어 데이터 프레임 리스트에 추가합니다.
dfs = []
for filename in all_files:
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 모든 데이터 프레임을 하나로 합칩니다.
merged_data = pd.concat(dfs, ignore_index=True)
merged_data.drop(columns=['project_name'], inplace=True)

In [9]:
surface_df = merged_data.loc[merged_data['수심(text)'] == '표층']
low_df = merged_data.loc[merged_data['수심(text)'] == '저층']

In [12]:
if not os.path.exists(folder_path):
    os.makedirs('./data/')
